In [ ]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import InputLayer, Conv2D, MaxPool2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.regularizers import l2

In [ ]:

train_dir = "/content/drive/MyDrive/data/HDA/Covid19-dataset/train"
test_dir = "/content/drive/MyDrive/data/HDA/Covid19-dataset/test"


In [ ]:
categories = ['Covid', 'Normal', 'Viral Pneumonia']
y_train = []
x_train = []
x_test = []
y_test = []
Batch_Size = 32
INIT_LR = 1e-4
EPOCHS = 10


In [ ]:
for category in categories:
    path = os.path.join(train_dir, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = cv2.imread(img_path)
        image = cv2.resize(image, (500, 500))
        image = image / 255
        x_train.append(image)
        y_train.append(categories.index(category))  # Use index as label


In [ ]:
for category in categories:
    path = os.path.join(test_dir, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = cv2.imread(img_path)
        image = cv2.resize(image, (500, 500))
        image = image / 255
        x_test.append(image)
        y_test.append(categories.index(category))  # Use index as label


In [ ]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)


In [ ]:

# Convert labels to categorical format
y_train_categorical = to_categorical(y_train, num_classes=3)
y_test_categorical = to_categorical(y_test, num_classes=3)


In [ ]:
  ## Creating a simple CNN model for classification
model = Sequential()
model.add(InputLayer(input_shape=(500, 500, 3)))
model.add(Conv2D(16, kernel_size=(2, 2), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(2, 2), activation='relu', kernel_regularizer=l2(1e-4)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))


In [ ]:
opt = Adam(learning_rate=INIT_LR)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
validation_steps = len(y_test) // Batch_Size
History = model.fit(
    np.array(x_train),
    np.array(y_train_categorical),
    epochs=EPOCHS,
    steps_per_epoch=len(x_train) // Batch_Size,
    batch_size=Batch_Size,
    validation_steps=validation_steps,
    validation_data=(np.array(x_test), np.array(y_test_categorical))
)

Epoch 1/10
8/8 [==============================] - 50s 6s/step - loss: 1.0024 - accuracy: 0.5430 - val_loss: 0.8060 - val_accuracy: 0.6094
Epoch 2/10
8/8 [==============================] - 47s 7s/step - loss: 0.7015 - accuracy: 0.7424 - val_loss: 0.6614 - val_accuracy: 0.6875
Epoch 3/10
8/8 [==============================] - 44s 5s/step - loss: 0.5623 - accuracy: 0.7598 - val_loss: 0.5693 - val_accuracy: 0.7500
Epoch 4/10
8/8 [==============================] - 44s 5s/step - loss: 0.4691 - accuracy: 0.7991 - val_loss: 0.5463 - val_accuracy: 0.7344
Epoch 5/10
8/8 [==============================] - 43s 5s/step - loss: 0.4210 - accuracy: 0.8821 - val_loss: 0.5304 - val_accuracy: 0.7500
Epoch 6/10
8/8 [==============================] - 43s 5s/step - loss: 0.4112 - accuracy: 0.8559 - val_loss: 0.5184 - val_accuracy: 0.7812
Epoch 7/10
8/8 [==============================] - 43s 5s/step - loss: 0.3552 - accuracy: 0.8734 - val_loss: 0.4817 - val_accuracy: 0.7344
Epoch 8/10
8/8 [==================